# Deplot notebook
> Creating an interactive environment for comparing stocks 



In [ ]:
#| default_exp deploy_project

In [ ]:
#| export

import buy_and_hold_vs_arima
import yfinance as yf
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.express as px
import pandas as pd
from waitress import serve
import numpy as np
import dash_bootstrap_components as dbc


In [ ]:
#| export

from buy_and_hold_vs_arima.arima_stats import create_auto_arima_prediction_future_2
from buy_and_hold_vs_arima.arima_stats import create_dataframe_with_series
from buy_and_hold_vs_arima.arima_stats import top_score
from buy_and_hold_vs_arima.arima_stats import get_top_players
from buy_and_hold_vs_arima.arima_stats import create_dataframe

In [ ]:
#| hide 

# how to import packages ! 
from buy_and_hold_vs_arima import arima_stats
arima_stats.foo("barak")

'Hello HAHAHAHbarak'

In [ ]:
#help(buy_and_hold_vs_arima)
#buy_and_hold_vs_arima.foo("hba")
dir(buy_and_hold_vs_arima)
#dir(np)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'arima_stats']

In [ ]:
barak = [1,2,3]
diker = [3,4,5]

In [ ]:
#| export 


#companies = ['AMZN','NFLX','GOOG']
companies = [
    "LUMI.TA",
    "DSCT.TA",
    "BEZQ.TA",
    "CEL.TA",
    "ESLT.TA",
    "NICE.TA",
    "TEVA.TA",
    "POLI.TA",
    "MZTF.TA",
    "FIBI.TA",
    "HARL.TA",
    "MGDL.TA",
    "CLIS.TA",
    "PHOE.TA",
    "MMHD.TA",
    "DRS.TA",
    "BSEN.TA",
    "HLAN.TA",
    "FTAL.TA",
    "DANE.TA",
    "ONE.TA",
    "MTRX.TA",
    "ALHE.TA",
    "UWAY.TA",
    "ICL.TA",
    "TA35.TA",
    "TA90.TA",
]

#companies = companies + ticker_stocks



tickers = [yf.Ticker(ticker).history( start='2021-12-10')['High'].rename(ticker) for ticker in companies]
df = pd.concat(tickers, axis=1)
df

for a in df.columns:
    print(a)
    
df[['LUMI.TA','ICL.TA']]
df['LUMI.TA'][df['LUMI.TA'] > 3000]

LUMI.TA
DSCT.TA
BEZQ.TA
CEL.TA
ESLT.TA
NICE.TA
TEVA.TA
POLI.TA
MZTF.TA
FIBI.TA
HARL.TA
MGDL.TA
CLIS.TA
PHOE.TA
MMHD.TA
DRS.TA
BSEN.TA
HLAN.TA
FTAL.TA
DANE.TA
ONE.TA
MTRX.TA
ALHE.TA
UWAY.TA
ICL.TA
TA35.TA
TA90.TA


,LUMI.TA,DSCT.TA,BEZQ.TA,CEL.TA,ESLT.TA,NICE.TA,TEVA.TA,POLI.TA,MZTF.TA,FIBI.TA,...,HLAN.TA,FTAL.TA,DANE.TA,ONE.TA,MTRX.TA,ALHE.TA,UWAY.TA,ICL.TA,TA35.TA,TA90.TA
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-23 00:00:00+02:00,3036.396093,1939.435791,457.284202,1726.0,53626.103053,97900.0,2669.0,2942.037955,10904.792969,11150.006583,...,20583.587891,32670.0,62505.157280,5520.862205,8681.778129,5178.287095,NaN,2571.331603,1955.449951,2299.679932
2021-12-26 00:00:00+02:00,3068.717385,1947.827357,461.258971,1749.0,53327.624895,98270.0,2688.0,2954.049848,10979.226913,11220.798522,...,21093.275817,32290.0,63429.716848,5426.970315,8734.254312,5191.267304,NaN,2587.649248,NaN,NaN
2021-12-27 00:00:00+02:00,3056.712158,1943.165471,469.763153,1748.0,53695.745676,97840.0,2692.0,2938.342446,10904.792712,11265.043945,...,20799.226213,31090.0,66091.298214,5541.518555,8630.221220,5277.494517,NaN,2598.814209,1964.819946,2302.199951
2021-12-28 00:00:00+02:00,3132.437718,1958.084246,478.359741,1777.0,53844.979079,97480.0,2740.0,2954.974445,11053.662257,11388.932346,...,20456.166402,31840.0,67725.597570,5633.533944,8727.811598,5459.221318,NaN,2688.132229,1995.949951,2331.409912
2021-12-29 00:00:00+02:00,3159.218514,1967.408418,481.595050,1771.0,54113.611892,97630.0,2714.0,3000.250484,11211.838921,11450.878245,...,20142.511170,31340.0,67576.175402,5597.854552,8678.096216,5485.182674,NaN,2674.391178,1995.260010,2315.979980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01 00:00:00+03:00,3125.142578,2047.730225,522.805664,1232.0,75761.531250,64510.0,3863.0,3362.548584,13743.335938,16261.638672,...,19750.000000,38400.0,31220.000000,4760.299316,7800.000000,2575.937988,5111.0,2085.234131,1873.150024,2012.650024
2023-10-02 00:00:00+03:00,3202.502638,2123.314331,534.488503,1265.0,78429.825938,66110.0,3910.0,3426.048901,13832.902047,16480.315797,...,20640.000000,38700.0,31700.000000,4898.120605,8049.000000,2679.567798,5750.0,2182.221786,1877.969971,2025.979980
2023-10-03 00:00:00+03:00,3184.650424,2104.418261,521.832092,1253.0,77960.126419,67210.0,3880.0,3404.220741,13852.805789,16609.533956,...,20560.000000,38550.0,32050.000000,4898.120714,8141.000000,2634.168004,5810.0,2176.283737,1869.569946,2022.380005


In [ ]:
df['LUMI.TA'].apply(lambda a: a > 3000)

Date
2021-12-12 00:00:00+02:00    False
2021-12-13 00:00:00+02:00    False
2021-12-14 00:00:00+02:00    False
2021-12-15 00:00:00+02:00    False
2021-12-16 00:00:00+02:00    False
                             ...  
2023-12-19 00:00:00+02:00    False
2023-12-20 00:00:00+02:00    False
2023-12-21 00:00:00+02:00    False
2023-12-24 00:00:00+02:00    False
2023-12-25 00:00:00+02:00    False
Name: LUMI.TA, Length: 504, dtype: bool

In [ ]:
#| export

# http://127.0.0.1:8050/

#df = px.data.gapminder()
#df.columns



external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        dcc.Markdown('''
        # Comparing Arima model and Buy and Hold strategy
        This is final project of master's degree in Haifa University 
        '''),
        dcc.Dropdown(
            id="dpdn2",
            value=["ICL.TA", "LUMI.TA"],
            multi=True,
            options=[{"label": x, "value": x} for x in df.columns],
        ),
         dbc.Row(
            
               # dcc.Graph(id="pie-graph", figure={}, className="six columns"),
                dcc.Graph(
                    id="my-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
           ),
            dbc.Row(
                dcc.Graph(
                    id="basic-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
            ),
        dbc.Row(
            # Values is the initial values
            # first arg and second arg are the overall range , third is the jumps 
                dcc.RangeSlider(0, len(df) - 1 , 1, count=1, value=[(len(df) - 1)*3//4, len(df) - 1] , id="range-inference") 
        ),
        dbc.Row(
            dcc.Graph(
                    id="inference-graph",
                    figure={},
                    clickData=None,
                    hoverData=None,
                    config={
                        "staticPlot": False,  # True, False
                        "scrollZoom": True,  # True, False
                        "doubleClick": "reset",  # 'reset', 'autosize' or 'reset+autosize', False
                        "showTips": False,  # True, False
                        "displayModeBar": True,  # True, False, 'hover'
                        "watermark": True,
                        # 'modeBarButtonsToAdd': ['pan2d','select2d'],
                    },
                    #className="six columns",
                ),
        ),
        
        
    ]
)


@app.callback(
    Output(component_id="my-graph", component_property="figure"),
    Input(component_id="dpdn2", component_property="value"),
)
def update_graph(country_chosen):
    #age_sex = titanic[["Age", "Sex"]]
    print(country_chosen)
    
    dff = df[country_chosen]
    predication_data = pd.concat([create_dataframe_with_series(create_auto_arima_prediction_future_2 , dff[country]).rename(columns={"High": country, "pred": country +"pred"}) for country in country_chosen])
   
    fig = px.line(
        predication_data,
        #dff,
        #x="Date[Days]",
        #y="Stock[agora]",
        #color="country",
        #custom_data=["country", "continent", "lifeExp", "pop"],
    )
    fig.update_traces(mode="lines+markers")
    return fig

@app.callback(
    Output(component_id="inference-graph", component_property="figure"),
    Input('range-inference', 'value'),
    Input(component_id="dpdn2", component_property="value"),
)
def update_output(slider_value,country_chosen):
    #print(value)
    dff = df[country_chosen]
    first_date_infer = dff.index[slider_value[0]]
    last_date_infer = dff.index[slider_value[1]]
    predication = pd.DataFrame()
    #concated_data = dff[country][first_date_infer:last_date_infer].copy()
    predication = pd.concat([create_dataframe_with_series(create_auto_arima_prediction_future_2 , dff[country][first_date_infer:last_date_infer].copy()).rename(columns={"High": country, "pred": country +"pred"}) for country in country_chosen])
    #predication.drop(country_chosen, axis=1)
    for country in country_chosen:
        predication[country + "True"] = dff[country][last_date_infer:]
    fig_global = px.line(
        predication,
        #x="Dates",
        #y=["Global Minimum Reg", "High"],
        #hover_data={"Dates": "|%B %d, %Y"},
        title="Inference BackTester",
    )

    fig_global.add_vline(
        x=first_date_infer, line_dash="dash", line_color="Blue"
    )
    fig_global.add_annotation(x=first_date_infer, text=str(first_date_infer))

    fig_global.add_vline(
        x=last_date_infer, line_dash="dash", line_color="Blue"
    )
    fig_global.add_annotation(x=last_date_infer, text=str(last_date_infer))
    
    ####################  what stocks to choose 
    #date = "2022-12-13"
    date = last_date_infer
    dict_best_to_invest = top_score(df,date,top=3)
    total_profit_of_best = sum(dict_best_to_invest.values())
    print("ARIMA expected profits are: " , total_profit_of_best)

    sum_arima_true = 0 
    if len(df.loc[date:]) > 30:
        for series in dict_best_to_invest:
            print(series) # The Stocks to invest 
            sum_arima_true += df[series].loc[date:].iloc[30] - df[series].loc[date:].iloc[0]
        print("ARIMA True profits are :" ,sum_arima_true )
    df_ta35 = create_dataframe(['TA35.TA'])
    if len(df_ta35.loc[date:]) > 30:
        ta35_profit = df_ta35.loc[date:].iloc[30] - df_ta35.loc[date:].iloc[0]
        print("TA35 Values profits are :" , ta35_profit.item())
    ####################
    
    return fig_global


@app.callback(
    Output(component_id="basic-graph", component_property="figure"),
    Input(component_id="dpdn2", component_property="value"),
)
def update_graph(country_chosen):
    #age_sex = titanic[["Age", "Sex"]]
    
    dff = df[country_chosen]
    fig = px.line(
        dff,
        #dff,
        #x="Date[Days]",
        #y="Stock[agora]",
        #color="country",
        #custom_data=["country", "continent", "lifeExp", "pop"],
    )
    fig.update_traces(mode="lines+markers")
    return fig






In [ ]:
#| hide
#if __name__ == "__main__":
#    app.run_server(debug=False)


In [ ]:
#| export 

def run_server(port=8050):
    serve(app.server, host="0.0.0.0", port=port, threads=2)


In [ ]:
#| export
if __name__ == "__main__":
    app.run_server(debug=False)
    #serve(app.server, host="0.0.0.0", port=8050, threads=2)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [25/Dec/2023 10:01:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2023 10:01:40] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_0m1677970696.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2023 10:01:40] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_0m1677970696.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2023 10:01:40] "GET /_dash-component-suites/dash/deps/react@16.v2_7_0m1677970696.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2023 10:01:40] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_0m1677970696.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2023 10:01:40] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_0m1677970696.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2023 10:01:40] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_5_0m1694087134.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2023 10:01

['ICL.TA', 'LUMI.TA']


/Users/barakdiker/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

127.0.0.1 - - [25/Dec/2023 10:01:41] "POST /_dash-update-component HTTP/1.1" 200 -
/Users/barakdiker/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/Users/barakdiker/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datet

ARIMA expected profits are:  2392.8396041107935


...

TA35 Values profits are : 66.6199951171875


/Users/barakdiker/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



ARIMA expected profits are:  3590.148319908607
HLAN.TA
ESLT.TA
FTAL.TA
ARIMA True profits are : -549.3635441476326


127.0.0.1 - - [25/Dec/2023 10:17:31] "POST /_dash-update-component HTTP/1.1" 200 -


TA35 Values profits are : -46.72998046875


In [ ]:
import nbdev; nbdev.nbdev_export()